# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/fabian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/fabian/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM-SAT - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [8]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [9]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [10]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [11]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node 'e5abeb'. Skipping!
Property 'summary' already exists in node 'b92034'. Skipping!
Property 'summary' already exists in node 'f5a20d'. Skipping!
Property 'summary' already exists in node '803efe'. Skipping!
Property 'summary' already exists in node '7b0d6b'. Skipping!
Property 'summary' already exists in node 'f89890'. Skipping!
Property 'summary' already exists in node '427389'. Skipping!
Property 'summary' already exists in node '5e37ee'. Skipping!
Property 'summary' already exists in node '9a20d8'. Skipping!
Property 'summary' already exists in node '2861ad'. Skipping!
Property 'summary' already exists in node '9bce39'. Skipping!
Property 'summary' already exists in node 'b3e971'. Skipping!
Property 'summary' already exists in node 'dfd8a0'. Skipping!
Property 'summary' already exists in node '5b3b37'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node 'e5abeb'. Skipping!
Property 'summary_embedding' already exists in node '7b0d6b'. Skipping!
Property 'summary_embedding' already exists in node 'f89890'. Skipping!
Property 'summary_embedding' already exists in node 'b3e971'. Skipping!
Property 'summary_embedding' already exists in node '427389'. Skipping!
Property 'summary_embedding' already exists in node 'f5a20d'. Skipping!
Property 'summary_embedding' already exists in node 'b92034'. Skipping!
Property 'summary_embedding' already exists in node '9a20d8'. Skipping!
Property 'summary_embedding' already exists in node '5b3b37'. Skipping!
Property 'summary_embedding' already exists in node '2861ad'. Skipping!
Property 'summary_embedding' already exists in node '9bce39'. Skipping!
Property 'summary_embedding' already exists in node '5e37ee'. Skipping!
Property 'summary_embedding' already exists in node 'dfd8a0'. Skipping!
Property 'summary_embedding' already exists in node '803efe'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 478)

We can save and load our knowledge graphs as follows.

In [12]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 40, relationships: 478)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [13]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [23]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [18]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,How do disbursements work in relation to acade...,"[Chapter 1 Academic Years, Academic Calendars,...",Disbursements are related to the academic year...,single_hop_specifc_query_synthesizer
1,Could you explain the significance of 34 CFR 6...,[Regulatory Citations Academic year minimums: ...,Regulatory citations specify that 34 CFR 668.3...,single_hop_specifc_query_synthesizer
2,What is the significance of Chapter 3 in relat...,[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work in a standard term ...,single_hop_specifc_query_synthesizer
3,How does the Federal Work-Study (FWS) program ...,[Non-Term Characteristics A program that measu...,The payment period is applicable to all Title ...,single_hop_specifc_query_synthesizer
4,What is the significance of Volume 8 in the co...,[both the credit or clock hours and the weeks ...,Volume 8 pertains to the principles of Direct ...,single_hop_specifc_query_synthesizer
5,How do disbursement rules and guidance under r...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,Disbursement rules and guidance under regulati...,multi_hop_abstract_query_synthesizer
6,how does the academic calendar relate to 34 CF...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The academic calendar defines the academic yea...,multi_hop_abstract_query_synthesizer
7,Considering the initial disbursement requireme...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,The school determines the timing of aid disbur...,multi_hop_abstract_query_synthesizer
8,How do Volume 2 and Volume 8 relate to the aca...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",Volume 2 discusses the definition of academic ...,multi_hop_specific_query_synthesizer
9,How do Appendix A and Appendix B relate to dis...,[<1-hop>\n\nboth the credit or clock hours and...,Appendix A provides guidance on calculating Pe...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [19]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '9785a3'. Skipping!
Property 'summary' already exists in node 'd8bd52'. Skipping!
Property 'summary' already exists in node 'ab89af'. Skipping!
Property 'summary' already exists in node '26ac88'. Skipping!
Property 'summary' already exists in node '2bc806'. Skipping!
Property 'summary' already exists in node '2c28ef'. Skipping!
Property 'summary' already exists in node '08d5af'. Skipping!
Property 'summary' already exists in node '3de3e1'. Skipping!
Property 'summary' already exists in node '6ed071'. Skipping!
Property 'summary' already exists in node '82dd83'. Skipping!
Property 'summary' already exists in node '7bd294'. Skipping!
Property 'summary' already exists in node 'c65183'. Skipping!
Property 'summary' already exists in node '5aff9e'. Skipping!
Property 'summary' already exists in node 'f80faa'. Skipping!


...

Property 'summary_embedding' already exists in node '9785a3'. Skipping!
Property 'summary_embedding' already exists in node 'd8bd52'. Skipping!
Property 'summary_embedding' already exists in node '6ed071'. Skipping!
Property 'summary_embedding' already exists in node '3de3e1'. Skipping!
Property 'summary_embedding' already exists in node '08d5af'. Skipping!
Property 'summary_embedding' already exists in node '2c28ef'. Skipping!
Property 'summary_embedding' already exists in node 'ab89af'. Skipping!
Property 'summary_embedding' already exists in node 'f80faa'. Skipping!
Property 'summary_embedding' already exists in node '7bd294'. Skipping!
Property 'summary_embedding' already exists in node '26ac88'. Skipping!
Property 'summary_embedding' already exists in node '5aff9e'. Skipping!
Property 'summary_embedding' already exists in node '2bc806'. Skipping!
Property 'summary_embedding' already exists in node '82dd83'. Skipping!
Property 'summary_embedding' already exists in node 'c65183'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [20]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,How do Academic Calendars influence the schedu...,"[Chapter 1 Academic Years, Academic Calendars,...",Academic Calendars are essential for managing ...,single_hop_specifc_query_synthesizer
1,Could you please explain the significance of 3...,[Regulatory Citations Academic year minimums: ...,Regulatory citations indicate that 34 CFR 668....,single_hop_specifc_query_synthesizer
2,What does Chapter 3 specify regarding clinical...,[Inclusion of Clinical Work in a Standard Term...,Chapter 3 states that clinical work conducted ...,single_hop_specifc_query_synthesizer
3,What are Non-Term Characteristics in education...,[Non-Term Characteristics A program that measu...,Non-Term Characteristics refer to programs tha...,single_hop_specifc_query_synthesizer
4,How do the academic year requirements in 34 CF...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The academic year requirements outlined in 34 ...,multi_hop_abstract_query_synthesizer
5,How do nonstandard terms and payment periods r...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Nonstandard terms are defined as terms that do...,multi_hop_abstract_query_synthesizer
6,Participation and requirements for practicum o...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Inclusion of clinical work in non-term program...,multi_hop_abstract_query_synthesizer
7,Include clinical work in standard term periods...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Inclusion of clinical work in standard term pe...,multi_hop_abstract_query_synthesizer
8,which volumes Volume 2 or Volume 7 is more imp...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The provided context does not specify the impo...,multi_hop_specific_query_synthesizer
9,Wht appendix A and B r relevnt for disbursmnt ...,[<1-hop>\n\nboth the credit or clock hours and...,Appendix A provides guidance on calculating Pe...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [24]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [25]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [27]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [29]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [30]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [31]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [32]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [33]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [34]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [35]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available include:\n\n- Direct Subsidized Loans  \n- Direct Unsubsidized Loans  \n- Student Direct PLUS Loans  \n- Parent Direct PLUS Loans on behalf of dependent students  \n\nAdditionally, loan types that were made under the Federal Family Education Loan (FFEL) Program before July 1, 2010, include:\n\n- Subsidized Federal Stafford Loans  \n- Unsubsidized Federal Stafford Loans  \n- Federal SLS Loans  \n- Federal PLUS Loans\n\nHowever, no new loans have been made under the FFEL Program since June 30, 2010.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [36]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [37]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

## LangSmith Evaluation

In [38]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'left-measure-4' at:
https://smith.langchain.com/o/bbbdc2e8-da22-470d-aa79-cd31d5ea3160/datasets/4ecc1253-944a-481e-b57c-d9c520493839/compare?selectedSessions=d9abcc93-304a-4303-b805-7f849d3f6b02




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,Can u plz tell me how the info in Chapter 2 ab...,I don't know.,None,The information in Chapter 2 explains that aca...,0,0,0,1.356108,47529fa8-291e-480d-8675-34c7874c7112,b590f39a-48a7-4319-b92d-f61fadd400d0
1,How do Volume 7 and Volume 8 relate to clinica...,"Based on the provided context:\n\n- Volume 7, ...",None,Volume 8 provides guidance on including clinic...,1,1,0,5.746603,451ceb71-ad57-465e-a2f3-29bcfa0145d6,b1a4eeed-fcfb-42e4-83ce-db1deeaddb8b
2,Wht appendix A and B r relevnt for disbursmnt ...,Appendix A and Appendix B are relevant for dis...,None,Appendix A provides guidance on calculating Pe...,1,1,0,5.205056,d1c4bc4c-5384-4808-8dfc-557ba53f7ec7,80e74245-d3dd-4a09-8748-739c9ee7d6aa
3,which volumes Volume 2 or Volume 7 is more imp...,"Based on the provided context, Volume 7 is foc...",None,The provided context does not specify the impo...,0,0,0,4.263636,9a4fceb9-a85b-4217-9ae4-38858e737209,ff11c446-6859-4c1c-b87b-e7c671e03fb4
4,Include clinical work in standard term periods...,Based on the provided context:\n\nClinical wor...,None,Inclusion of clinical work in standard term pe...,1,1,0,4.569155,985b6abe-b276-49e7-8d26-3c87329196bd,05d3102c-7d29-4d61-bc87-d69dfdf59874
5,Participation and requirements for practicum o...,Based on the provided context:\n\n- Periods of...,None,Inclusion of clinical work in non-term program...,1,1,0,7.334026,e05a7f5e-6746-4c7e-8826-e29c32bcb6dd,bbc079bd-5a14-4c09-94de-3b9ef436044b
6,How do nonstandard terms and payment periods r...,Based on the provided context:\n\n- Nonstandar...,None,Nonstandard terms are defined as terms that do...,1,1,0,6.113790,3e3e163a-bbcf-402d-9222-6f13cf7f27a4,edc04a4d-eef0-42e9-8459-d963531f37e6
7,How do the academic year requirements in 34 CF...,The academic year requirements in 34 CFR 668.3...,None,The academic year requirements outlined in 34 ...,1,1,0,3.045825,1d208410-aaf3-4ed1-92e2-73fc65ee5c32,e61b8c46-ec8d-452c-82ae-8b6ad5f14cd5
8,What are Non-Term Characteristics in education...,Non-Term Characteristics in educational progra...,None,Non-Term Characteristics refer to programs tha...,1,1,0,2.935126,adf81e87-887a-45f9-88b9-de3b6fd80320,20cad60b-7511-4767-9a5e-46423d8788d5
9,What does Chapter 3 specify regarding clinical...,Chapter 3 specifies that periods of medical an...,None,Chapter 3 states that clinical work conducted ...,1,1,0,3.641530,1ec7650d-3ddb-45dc-8772-d22c64dd6d74,1364980f-19a6-4137-b6e3-652705d779ed


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [39]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [40]:
rag_documents = docs

In [41]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

In [42]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

In [43]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [44]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [45]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [46]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question—it's really important to understand the types of loans available to help manage educational expenses. Based on the information provided, there are several kinds of federal student loans available:\n\n1. **Direct Subsidized Loans:** These are loans for students with financial need, and the government pays the interest while the student is in school at least half-time. They're limited to the student's financial need, which is calculated based on the Cost of Attendance (COA) minus other aid.\n\n2. **Direct Unsubsidized Loans:** These loans are available to students regardless of financial need. Interest accrues while the student is in school, but these loans can be used to cover unmet financial need or to replace other aid not received.\n\n3. **Direct PLUS Loans:** These loans are available to parents of dependent students, assuming they meet eligibility requirements. There’s no fixed loan limit for PLUS loans, but the total cannot exceed the student's COA min

Finally, we can evaluate the new chain on the same test set!

In [47]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'glossy-start-67' at:
https://smith.langchain.com/o/bbbdc2e8-da22-470d-aa79-cd31d5ea3160/datasets/4ecc1253-944a-481e-b57c-d9c520493839/compare?selectedSessions=f7b07005-f858-4fd5-aea4-f6b67eacfc96




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,Can u plz tell me how the info in Chapter 2 ab...,Thank you for your thoughtful question—it’s cl...,None,The information in Chapter 2 explains that aca...,1,1,1,16.826207,47529fa8-291e-480d-8675-34c7874c7112,14a849c8-bcf2-461e-8f73-ac85e19742a1
1,How do Volume 7 and Volume 8 relate to clinica...,Thank you for your thoughtful question. Based ...,None,Volume 8 provides guidance on including clinic...,1,1,1,5.703938,451ceb71-ad57-465e-a2f3-29bcfa0145d6,ccc2343b-051e-4c32-9485-5fd7d59e6eed
2,Wht appendix A and B r relevnt for disbursmnt ...,Thank you for your thoughtful question about t...,None,Appendix A provides guidance on calculating Pe...,0,0,1,5.171669,d1c4bc4c-5384-4808-8dfc-557ba53f7ec7,665a919d-9228-4aee-be4d-443dbaed6341
3,which volumes Volume 2 or Volume 7 is more imp...,Thank you for your thoughtful question. Based ...,None,The provided context does not specify the impo...,1,0,1,4.237880,9a4fceb9-a85b-4217-9ae4-38858e737209,d8abaac9-67a1-43b1-ace8-c13a50521696
4,Include clinical work in standard term periods...,Thank you for your thoughtful question. Based ...,None,Inclusion of clinical work in standard term pe...,1,1,1,6.190225,985b6abe-b276-49e7-8d26-3c87329196bd,126a63b1-22e4-435d-8579-6de3c27d449d
5,Participation and requirements for practicum o...,Thank you for your thoughtful question. I unde...,None,Inclusion of clinical work in non-term program...,1,0,1,7.863446,e05a7f5e-6746-4c7e-8826-e29c32bcb6dd,8be6458f-232b-433e-8ba4-ea5564c60faa
6,How do nonstandard terms and payment periods r...,Thank you for your thoughtful question. Based ...,None,Nonstandard terms are defined as terms that do...,1,1,1,9.981392,3e3e163a-bbcf-402d-9222-6f13cf7f27a4,08a2e981-bd4e-4f2c-8f37-8b796c226608
7,How do the academic year requirements in 34 CF...,Thank you for your thoughtful question. Based ...,None,The academic year requirements outlined in 34 ...,1,0,1,3.952221,1d208410-aaf3-4ed1-92e2-73fc65ee5c32,bf563976-c149-4066-9c13-d452d665a2a3
8,What are Non-Term Characteristics in education...,Thank you for your thoughtful question about N...,None,Non-Term Characteristics refer to programs tha...,1,1,1,5.011026,adf81e87-887a-45f9-88b9-de3b6fd80320,6f08764d-6ee5-4b48-bd4c-d805639d16b6
9,What does Chapter 3 specify regarding clinical...,Thank you for your thoughtful question. Based ...,None,Chapter 3 states that clinical work conducted ...,1,1,1,4.713627,1ec7650d-3ddb-45dc-8772-d22c64dd6d74,9cbfeeec-628e-4c4a-a788-d662b2f5b6ed


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.